<a href="https://colab.research.google.com/github/profteachkids/subtitle_generator/blob/main/subtitle_timeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython
!pip install bidict
!pip install torchaudio

     |████████████████████████████████| 2.3 MB 5.4 MB/s 
     |████████████████████████████████| 1.9 MB 5.3 MB/s 
     |████████████████████████████████| 831.4 MB 6.3 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.9.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.9.1 which is incompatible.


In [2]:
!wget https://github.com/profteachkids/subtitle_generator/raw/main/en_v5.jit


--2021-10-03 13:29:09--  https://github.com/profteachkids/subtitle_generator/raw/main/en_v5.jit
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/profteachkids/subtitle_generator/main/en_v5.jit [following]
--2021-10-03 13:29:09--  https://media.githubusercontent.com/media/profteachkids/subtitle_generator/main/en_v5.jit
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117217582 (112M) [application/octet-stream]
Saving to: ‘en_v5.jit’

en_v5.jit           100%[===================>] 111.79M   178MB/s    in 0.6s    

2021-10-03 13:29:13 (178 MB/s) - ‘en_v5.jit’ saved [117217582/11721758

In [ ]:
import numpy as np
from lxml import etree
import subprocess as sp
import os
import torch
import pandas as pd
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from bidict import bidict
import string
from glob import glob
import re
import shutil
from itertools import groupby

In [ ]:
class Decoder():
    def __init__(self,labels):
        self.labels = labels
        self.blank_idx = self.labels.index('_')
        self.space_idx = self.labels.index(' ')

    def process(self,
                probs, wav_len, word_align):
        assert len(self.labels) == probs.shape[1]
        for_string = []
        argm = torch.argmax(probs, axis=1)
        align_list = [[]]
        for j, i in enumerate(argm):
            if i == self.labels.index('2'):
                try:
                    prev = for_string[-1]
                    for_string.append('$')
                    for_string.append(prev)
                    align_list[-1].append(j)
                    continue
                except:
                    for_string.append(' ')
                    warnings.warn('Token "2" detected a the beginning of sentence, omitting')
                    align_list.append([])
                    continue
            if i != self.blank_idx:
                for_string.append(self.labels[i])
                if i == self.space_idx:
                    align_list.append([])
                else:
                    align_list[-1].append(j)

        string = ''.join([x[0] for x in groupby(for_string)]).replace('$', '').strip()

        align_list = list(filter(lambda x: x, align_list))

        if align_list and wav_len and word_align:
            align_dicts = []
            linear_align_coeff = wav_len / len(argm)
            to_move = min(align_list[0][0], 1.5)
            for i, align_word in enumerate(align_list):
                if len(align_word) == 1:
                    align_word.append(align_word[0])
                align_word[0] = align_word[0] - to_move
                if i == (len(align_list) - 1):
                    to_move = min(1.5, len(argm) - i)
                    align_word[-1] = align_word[-1] + to_move
                else:
                    to_move = min(1.5, (align_list[i+1][0] - align_word[-1]) / 2)
                    align_word[-1] = align_word[-1] + to_move

            for word, timing in zip(string.split(), align_list):
                align_dicts.append({'word': word,
                                    'start_ts': round(timing[0] * linear_align_coeff, 2),
                                    'end_ts': round(timing[-1] * linear_align_coeff, 2)})

            return string, align_dicts
        return string

    def __call__(self,
                 probs: torch.Tensor,
                 wav_len: float = 0,
                 word_align: bool = False):
        return self.process(probs, wav_len, word_align)

torch.set_grad_enabled(False)

model = torch.jit.load("en_v5.jit", map_location=torch.device('cpu'))
model.eval()
decoder=Decoder(model.labels)

In [ ]:
text = """after i was betrayed
by my minecraft friend
and surrounded by my
enemies on the
public smp
in the last part
i was done being
pushed around and
decided to muster up
my guts and fight back
now unlike what
happened in a previous
video this time didn't go
as well...
my armor was breaking
i was running low on
materials so i did what
anyone in my position
would do
hide my gear and run
that plan would've been
so much better if i still
didn't die 
but its safe to say that
my enemies still aren't
the top 10 happiest people
on the public smp right now
help me in this war
server ip in Discord"""

text = re.sub(r"[,.!?-] ", " ", text, 0, re.MULTILINE)
text = text.lower()
text = text.split()


In [ ]:
DEVNULL = open(os.devnull, "w")

def ffmpeg_load_audio(
    filename,
    sr=44100,
    mono=False,
    normalize=True,
    in_type=np.int16,
    out_type=np.float32,
):
    channels = 1 if mono else 2
    format_strings = {
        np.float64: "f64le",
        np.float32: "f32le",
        np.int16: "s16le",
        np.int32: "s32le",
        np.uint32: "u32le",
    }
    format_string = format_strings[in_type]
    command = [
        "ffmpeg",
        "-i",
        filename,
        "-f",
        format_string,
        "-acodec",
        "pcm_" + format_string,
        "-ar",
        str(sr),
        "-ac",
        str(channels),
        "-",
    ]
    p = sp.Popen(command, stdout=sp.PIPE, stderr=DEVNULL, bufsize=4096)
    bytes_per_sample = np.dtype(in_type).itemsize
    frame_size = bytes_per_sample * channels
    chunk_size = frame_size * sr  # read in 1-second chunks
    raw = b""
    with p.stdout as stdout:
        while True:
            data = stdout.read(chunk_size)
            if data:
                raw += data
            else:
                break
    audio = np.fromstring(raw, dtype=in_type).astype(out_type)
    if channels > 1:
        audio = audio.reshape((-1, channels)).transpose()
    if audio.size == 0:
        return audio, sr
    if issubclass(out_type, np.floating):
        if normalize:
            peak = np.abs(audio).max()
            if peak > 0:
                audio /= peak
        elif issubclass(in_type, np.integer):
            audio /= np.iinfo(in_type).max
    return audio, sr

def to_text(f):
    print(f)
    audio, sr = ffmpeg_load_audio(
        f, sr=16000, in_type=np.float32, out_type=np.float32
    )
    audio = np.expand_dims(audio.astype(np.float32)[0], 0)

    res = []
    batch_size =  3*sr
    N = audio.size // batch_size

    sections = np.array_split(audio, N, 1)
    section_start=0
    for i, section in enumerate(sections):
        input=torch.from_numpy(section)
        output = model(input)[0]
        decoded=decoder(output.cpu(), section.size, word_align=True)
        if len(decoded)==2:
            s,dlist = decoded
            for d in dlist:
                d["start_ts"] = (d["start_ts"] + section_start)/sr
                d["end_ts"] = (d["end_ts"] +  section_start)/sr
                print(d["word"], d["start_ts"], d["end_ts"])
                res.append(d)
            section_start+=section.size


    df = pd.DataFrame(res)
    df.to_csv("voice_times.csv")

    voice_words = df["word"].values
    print(" ".join(voice_words))

def align():
    df = pd.read_csv("voice_times.csv")
    voice_words = df["word"].values
    start_times = df["start_ts"].values
    end_times = df["end_ts"].values

    all_words = set(text).union(set(voice_words))

    word_dict = bidict(zip(all_words, range(len(all_words))))
    word_dict["-"] = "-"
    voice = [word_dict[word] for word in voice_words]
    script = [word_dict[word] for word in text]
    align = pairwise2.align.globalxx(voice, script, gap_char =['-'], one_alignment_only=True)[0]
    matches=[]
    mismatches=[]
    mismatch_voice=word_dict.inverse[align.seqA[0]]   
    mismatch_text=word_dict.inverse[align.seqB[0]]
    v_end_pos=0
    v_start_pos=0
    t_end_pos=0
    t_start_pos=0
    for i,(v, t) in enumerate(zip(align.seqA[1:], align.seqB[1:])):
        print(word_dict.inverse[v], word_dict.inverse[t])
        if v==t:
            mismatches.append([t_start_pos, t_end_pos, start_times[v_start_pos], end_times[v_end_pos], mismatch_voice, mismatch_text])
            v_end_pos+=1
            v_start_pos=v_end_pos
            t_end_pos+=1
            t_start_pos=t_end_pos
            mismatch_voice=word_dict.inverse[v]
            mismatch_text=word_dict.inverse[t]
        else:
            if word_dict.inverse[v] != '-' :
                v_end_pos+=1
            if word_dict.inverse[t] != '-' :
                t_end_pos+=1
            mismatch_voice+=' ' + word_dict.inverse[v]
            mismatch_text+=' ' + word_dict.inverse[t]


    mismatches.append([t_start_pos, t_end_pos, start_times[v_start_pos], end_times[v_end_pos], mismatch_voice, mismatch_text])

    idx=[]
    times=[]
    for t_start_pos, t_end_pos, start_time, end_time, v, t in mismatches:
        print('-'*30)
        print(v)
        print(t)
        print(t_start_pos,t_end_pos, start_time, end_time)
        d={}

        t2 = t.translate(t.maketrans(string.punctuation, " " * len(string.punctuation)))
        t2 = " ".join(t2.split())

        idx.append(t_start_pos)
        times.append(start_time)

    idx.append(t_end_pos+1)
    times.append(end_time)

    np.savez('it',idx=np.array(idx),time=np.array(times))
    it = np.load('it.npz')
    print(it['idx'], len(it['idx']))
    print(it['time'], len(it['time']))

def make_copies():
    it = np.load('it.npz')
    idx=it['idx']
    times=it['time']
    print(times, len(times))
    path_out='E:\\Blender\\BlenderVideo\\'
    path_in='E:\\Blender\\BlenderOut\\'

    i=0
    subtitle_images = glob(path_in+'*.png')
    print(subtitle_images)
    for subtitle_image in subtitle_images:
        name, subtitle_n, w_start, w_end = subtitle_image.split('_')
        *_, name = name.split('\\')
        w_end,_ = w_end.split('.')
        w_start, w_end, subtitle_n =int(w_start), int(w_end), int(subtitle_n)
        t_start, t_end = np.interp([w_start, w_end], idx, times)
        f_start, f_end = int(t_start*10), int(t_end*10)
        print(name, w_start, w_end, t_start, t_end, f_start, f_end)
        for copy_n in range(f_end-f_start+1):
            
            name=f'{i:04d}'.translate(str.maketrans('0123456789','abcdefghij'))
            shutil.copyfile(subtitle_image, path_out+name+'.png')
            i+=1

def set_timeline():
    it = np.load('it.npz')
    idx=it['idx']
    times=it['time']


    tree = etree.parse('Timeline 1.xml')
    clips=tree.xpath('//video//clipitem')
    for clip in clips:
        name=clip.xpath('name/text()')[0]
        start=clip.xpath('start')[0]
        end=clip.xpath('end')[0]
        name, subtitle_n, w_start, w_end = name.split('_')
        *_, name = name.split('\\')
        w_end,_ = w_end.split('.')
        w_start, w_end, subtitle_n =int(w_start), int(w_end), int(subtitle_n)
        t_start, t_end = np.interp([w_start, w_end], idx, times)
        start.text=str(int(t_start*60))
        end.text=str(int(t_end*60))

    f = open('subtitle_timeline.xml', 'wb')
    f.write(etree.tostring(tree, pretty_print=True))
    f.close()



In [ ]:
to_text("audio.mov")
align()    
set_timeline()

audio.mov


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


after 0.0 0.19866375
i 0.19866375 0.43706124999999996
escaped 0.43706124999999996 0.79465625
the 0.79465625 0.9535875
dark 0.9535875 1.2317175
endanger 1.3111825 1.74824375
tunne 1.74824375 2.185305
in 2.185305 2.38396875
the 2.38396875 2.5429
last 2.5429 2.70183125
part 2.70183125 2.8607625
i 2.8607625 3.059426875
a 3.178625 3.2580887499999998
breakthrough 3.2580887499999998 3.854069375
i 4.0129975 4.2116575
shouldget 4.2116575 4.5295137500000004
better 4.5295137500000004 4.728174375000001
weaps 4.728174375000001 5.204958749999999
'sad 5.443350625 5.880403125
of 5.880403125 6.03933125
running 6.03933125 6.317455625000001
won 6.3571875 6.87370375
after 6.87370375 7.1120962500000005
dying 7.191559999999999 7.668344374999999
a 7.668344374999999 7.867005
few 7.867005 8.065665
more 8.065665 8.264325000000001
times 8.264325000000001 8.542449375
i 8.621913124999999 8.820573125
wouldliketo 8.820573125 9.416553749999999
i 9.65494625 9.893338125
finally 9.9728025 10.409854375
obtained 10.409854